# Portfolio Performance vs. Benchmark


### Step 1: Import Libraries


In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import glob
import logging
import warnings

# System libraries
import bt
from py.quantstats_fix import *
from py.utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/__init__.py
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py...
✓ Found 'plot_timeserie

NameError: name 'qs' is not defined

In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*inplace.*")
warnings.filterwarnings("ignore", message=".*DataFrame.prod.*")
warnings.filterwarnings("ignore", message=".*deprecated.*")

# Disable matplotlib font manager logging
logging.getLogger('matplotlib.font_manager').disabled = True

### Step 2: Define Parameters 


#### Dates


In [ ]:
# Define the date range
initial_end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day

# Check if portfolio file exists for the initial end date
expected_portfolio_file = f'portfolios/portfolio-{datetime.date(initial_end_date)}.xlsx'

if os.path.exists(expected_portfolio_file):
    end_date = initial_end_date
    print(f"Using portfolio file date: {expected_portfolio_file}")
else:
    # If the expected file doesn't exist, use the most recent portfolio file date
    portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
    if portfolio_files:
        # Get the most recent file and extract date from filename
        latest_file = max(portfolio_files, key=os.path.getmtime)
        # Extract date from filename (assumes format: portfolio-YYYY-MM-DD.xlsx)
        import re
        date_match = re.search(r'portfolio-(\d{4}-\d{2}-\d{2})\.xlsx', latest_file)
        if date_match:
            file_date_str = date_match.group(1)
            end_date = pd.to_datetime(file_date_str).to_pydatetime()
            print(f"Using date from existing portfolio file: {latest_file}")
        else:
            end_date = initial_end_date
            print(f"Could not extract date from filename, using default: {initial_end_date}")
    else:
        end_date = initial_end_date
        print(f"No portfolio files found, using default: {initial_end_date}")

start_date = end_date - timedelta(days=10*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Using date from existing portfolio file: portfolios/portfolio-2025-06-06.xlsx
Date Range: 2015-06-09 to 2025-06-06
Time span: 3650 days (10.00 years)


#### Report File


In [ ]:
# Get the most recent portfolio file
portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
if portfolio_files:
    # Sort files by modification time (most recent first)
    output_file = max(portfolio_files, key=os.path.getmtime)
    print(f"Using most recent portfolio file: {output_file}")
else:
    # Fallback to current date if no files found
    output_file = f'portfolios/portfolio-{datetime.date(end_date)}.xlsx'
    print(f"No portfolio files found. Using: {output_file}")

Using most recent portfolio file: portfolios/portfolio-2025-06-06.xlsx


#### Benchmark


In [ ]:
# Long-term benchmark
benchmark_long = pd.read_excel(output_file, sheet_name="benchmark_long_term")['Benchmark'].values[0]
print(f"Long-term Benchmark: {benchmark_long}")

# Short-term benchmark
benchmark_short = pd.read_excel(output_file, sheet_name="benchmark_short_term")['Benchmark'].values[0]
print(f"Short-term Benchmark: {benchmark_short}")

Long-term Benchmark: YYY
Short-term Benchmark: DSI


#### Risk-free rate (T-bill, %)


In [ ]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: 0.0424 -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)


In [ ]:
# Long-term portfolio data
portfolio_long_df = pd.read_excel(output_file, sheet_name="long_term")
portfolio_long_df['Weight'] = portfolio_long_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_long_tickers = portfolio_long_df["Ticker"].tolist()

# Short-term portfolio data
portfolio_short_df = pd.read_excel(output_file, sheet_name="short_term")
portfolio_short_df['Weight'] = portfolio_short_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_short_tickers = portfolio_short_df["Ticker"].tolist()

print("Long-term Portfolio Tickers:", portfolio_long_tickers)
display(portfolio_long_df)
print("Short-term Portfolio Tickers:", portfolio_short_tickers)
display(portfolio_short_df)

Long-term Portfolio Tickers: ['MMC', 'AMAT', 'TMUS', 'MRK', 'APO']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,52W Low,50 Day Avg,200 Day Avg,Short Ratio,Short % of Float,Strategy,Weight,Expected Return,Standard Deviation,Sharpe Ratio
0,MMC,2025-06-02,"Marsh & McLennan Companies, Inc.",Financial Services,Insurance Brokers,United States,https://www.marshmclennan.com,115130826752,136148770816,491821141,...,205.17,229.54,225.30,2.62,0.01,Markowitz,0.318087,0.140449,0.212929,0.659604
1,AMAT,2025-06-02,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,United States,https://www.appliedmaterials.com,125791559680,125714595840,799392549,...,123.74,151.52,173.39,2.58,0.02,Markowitz,0.228911,0.181314,0.412511,0.439538
2,TMUS,2025-06-02,"T-Mobile US, Inc.",Communication Services,Telecom Services,United States,https://www.t-mobile.com,275005997056,384693796864,452736028,...,171.18,251.20,232.97,3.11,0.03,Markowitz,0.223672,0.130260,0.275833,0.472243
3,MRK,2025-06-02,"Merck & Co., Inc.",Healthcare,Drug Manufacturers - General,United States,https://www.merck.com,192947535872,218628636672,2507088934,...,73.31,81.27,97.40,2.49,0.02,Markowitz,0.145330,0.095043,0.220747,0.430553
4,APO,2025-06-02,"Apollo Global Management, Inc.",Financial Services,Asset Management,United States,https://www.apollo.com/institutional/homepage,74688552960,94705582080,446479870,...,95.11,133.28,143.86,7.75,0.05,Markowitz,0.084000,0.135080,0.379183,0.356240


Short-term Portfolio Tickers: ['MSFT', 'NOC']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,Short Period,Long Period,Articles In Last Week,Company News Score,Bearish Percent,Bullish Percent,Average Sentiment Score,Sector Average Bullish Percent,Sector Average News Score,Weight
0,MSFT,2025-06-02,Microsoft Corporation,Technology,Software - Infrastructure,United States,https://www.microsoft.com,3421643997184,3447046799360,7422063978,...,85,125,42,0.5954,0.0476,0.619,0.1909,0.4767,0.5650,0.5
1,NOC,2025-06-02,Northrop Grumman Corporation,Industrials,Aerospace & Defense,United States,https://www.northropgrumman.com,69771976704,86213066752,135162959,...,85,125,3,0.6516,0.0000,1.000,0.3031,0.5147,0.5657,0.5


### Step 4: Download Returns


In [ ]:
# Long-term stock returns
stock_long_quotes = load_and_filter_data('data/daily_stock_quotes.csv', portfolio_long_tickers, start_date, end_date)
stock_long_returns = np.log(stock_long_quotes / stock_long_quotes.shift(1)).dropna()

# Short-term stock returns
stock_short_quotes = load_and_filter_data('data/daily_stock_quotes.csv', portfolio_short_tickers, start_date, end_date)
stock_short_returns = np.log(stock_short_quotes / stock_short_quotes.shift(1)).dropna()

# Long-term benchmark returns
benchmark_long_quotes = load_and_filter_data('data/daily_benchmark_quotes.csv', benchmark_long, start_date, end_date)
benchmark_long_returns = np.log(benchmark_long_quotes / benchmark_long_quotes.shift(1)).dropna()

# Short-term benchmark returns
benchmark_short_quotes = load_and_filter_data('data/daily_benchmark_quotes.csv', benchmark_short, start_date, end_date)
benchmark_short_returns = np.log(benchmark_short_quotes / benchmark_short_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for Long-term assets:")
sharpes_long = {col: qs.stats.sharpe(stock_long_returns[col]) for col in stock_long_returns.columns}
for ticker, sharpe in sharpes_long.items():
    print(f"{ticker}: {sharpe:.4f}")

print("\nSharpe Ratios for Short-term assets:")
sharpes_short = {col: qs.stats.sharpe(stock_short_returns[col]) for col in stock_short_returns.columns}
for ticker, sharpe in sharpes_short.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_long_returns.head())
display(stock_short_returns.head())


Found 5 of 5 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 2 of 2 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for Long-term assets:
AMAT: 0.5381
APO: 0.6056
MMC: 0.7671
MRK: 0.2862
TMUS: 0.7004

Sharpe Ratios for Short-term assets:
MSFT: 0.9075
NOC: 0.5229


,AMAT,APO,MMC,MRK,TMUS
Date,,,,,
2015-06-10,0.006776,0.002220,0.016007,0.012978,-0.003169
2015-06-11,-0.013028,0.002215,0.008008,-0.004307,-0.014117
2015-06-12,-0.008014,-0.005917,-0.003796,-0.017905,0.022807
2015-06-15,0.004587,-0.005206,-0.005017,-0.013024,-0.001049
2015-06-16,0.005136,-0.000746,0.003414,0.010335,-0.002891


,MSFT,NOC
Date,,
2015-06-10,0.021001,0.008453
2015-06-11,-0.003718,0.013858
2015-06-12,-0.010233,0.001819
2015-06-15,-0.010846,-0.006856
2015-06-16,0.007831,0.009542


### Step 5: Split Data into Training and Testing Sets

In [ ]:
total_rows = len(stock_long_quotes)
train_size = int(0.8 * total_rows)
training_set = stock_long_quotes.iloc[:train_size]
test_set = stock_long_quotes.iloc[train_size:]

print(f"Training: {len(training_set)} rows ({len(training_set)/total_rows:.1%}) | Testing: {len(test_set)} rows ({len(test_set)/total_rows:.1%})")
print(f"Training period: {training_set.index[0]} to {training_set.index[-1]}")
print(f"Testing period: {test_set.index[0]} to {test_set.index[-1]}")

Training: 2010 rows (80.0%) | Testing: 503 rows (20.0%)
Training period: 2015-06-09 00:00:00 to 2023-06-01 00:00:00
Testing period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00


### Step 5: Plot Return Comparisons


In [ ]:
# Long-term portfolio returns
portfolio_long_weights = portfolio_long_df.set_index('Ticker')['Weight'].to_dict()
weighted_long_returns = pd.DataFrame()

print("\nLong-term Portfolio Weights:")
for ticker in portfolio_long_tickers:
    weight = portfolio_long_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_long_returns.columns:
        weighted_long_returns[ticker] = stock_long_returns[ticker] * weight

# Sum across all weighted returns to get the long-term portfolio return
portfolio_long_return = weighted_long_returns.sum(axis=1)

# Short-term portfolio returns
portfolio_short_weights = portfolio_short_df.set_index('Ticker')['Weight'].to_dict()
weighted_short_returns = pd.DataFrame()

print("\nShort-term Portfolio Weights:")
for ticker in portfolio_short_tickers:
    weight = portfolio_short_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_short_returns.columns:
        weighted_short_returns[ticker] = stock_short_returns[ticker] * weight

# Sum across all weighted returns to get the short-term portfolio return
portfolio_short_return = weighted_short_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight_long = 1/len([t for t in portfolio_long_tickers if t in stock_long_returns.columns])
equal_weighted_long_returns = pd.DataFrame()

for ticker in portfolio_long_tickers:
    if ticker in stock_long_returns.columns:
        equal_weighted_long_returns[ticker] = stock_long_returns[ticker] * equal_weight_long
        
equal_weight_long_return = equal_weighted_long_returns.sum(axis=1)

# Create equal-weight portfolio for short-term comparison
equal_weight_short = 1/len([t for t in portfolio_short_tickers if t in stock_short_returns.columns])
equal_weighted_short_returns = pd.DataFrame()

for ticker in portfolio_short_tickers:
    if ticker in stock_short_returns.columns:
        equal_weighted_short_returns[ticker] = stock_short_returns[ticker] * equal_weight_short
        
equal_weight_short_return = equal_weighted_short_returns.sum(axis=1)

print("\nLong-term Portfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_long_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_long_return):.4f}")

print("\nShort-term Portfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_short_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_short_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_long_return).cumprod().plot(label='Weighted Long-term Portfolio')
(1 + equal_weight_long_return).cumprod().plot(label='Equal Weight Long-term')
(1 + benchmark_long_returns).cumprod().plot(label=benchmark_long)
plt.legend()
plt.title('Long-term Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
(1 + portfolio_short_return).cumprod().plot(label='Weighted Short-term Portfolio')
(1 + equal_weight_short_return).cumprod().plot(label='Equal Weight Short-term')
(1 + benchmark_short_returns).cumprod().plot(label=benchmark_short)
plt.legend()
plt.title('Short-term Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()



Long-term Portfolio Weights:
MMC: 31.81%
AMAT: 22.89%
TMUS: 22.37%
MRK: 14.53%
APO: 8.40%

Short-term Portfolio Weights:
MSFT: 50.00%
NOC: 50.00%

Long-term Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 0.8386
Sharpe Ratio (Equal-Weight): 0.8155

Short-term Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 0.9055
Sharpe Ratio (Equal-Weight): 0.9055


### Step 6: Generate Reports


In [ ]:
# Filter portfolio_return and benchmark_returns to match test_set dates
test_start_date = test_set.index[0]
test_end_date = test_set.index[-1]

print(f"Test period: {test_start_date} to {test_end_date}")

Test period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00


#### **Long-Term Portfolio Performance**

In [ ]:
print("\n" + "="*60)
print("LONG-TERM PORTFOLIO BACKTESTING")
print("="*60)

# Create weights dictionary for long-term portfolio
weights_long = {}
for _, row in portfolio_long_df.iterrows():
    ticker = row['Ticker']
    weight = row['Weight']
    weights_long[ticker] = weight

print("Long-term Portfolio weights:")
for ticker, weight in weights_long.items():
    print(f"{ticker}: {weight:.4f}")

# Create price data for bt (using test set)
all_quotes_long = test_set.copy()
all_quotes_long[benchmark_long] = benchmark_long_quotes[benchmark_long].loc[test_start_date:test_end_date]

# Create bt strategies for long-term
portfolio_strategy_long = bt.Strategy('Long-term Portfolio', 
                                     [
                                         bt.algos.RunEveryNPeriods(66, offset=66),  # Quarterly rebalancing
                                         bt.algos.SelectAll(),
                                         bt.algos.WeighSpecified(**weights_long),       
                                         bt.algos.Rebalance()
                                     ])

benchmark_strategy_long = bt.Strategy(f'{benchmark_long}',
                                     [
                                         bt.algos.RunEveryNPeriods(66, offset=66),
                                         bt.algos.SelectThese([benchmark_long]),
                                         bt.algos.WeighEqually(),
                                         bt.algos.Rebalance()
                                     ])

# Create backtests for long-term
portfolio_test_long = bt.Backtest(portfolio_strategy_long, all_quotes_long[portfolio_long_tickers])
benchmark_test_long = bt.Backtest(benchmark_strategy_long, all_quotes_long[[benchmark_long]])

# Run backtests for long-term
result_long = bt.run(portfolio_test_long, benchmark_test_long)
result_long.set_riskfree_rate(risk_free_rate)

print("\nLong-term Backtest Results:")
result_long.display()

# Plot long-term results
result_long.plot(figsize=(12, 8),
                ylabel='Cumulative Returns',
                title='Long-term Portfolio vs Benchmark Performance',
                legend=True,
                grid=True)

# Generate long-term QuantStats reports
portfolio_return_long_test = portfolio_long_return.loc[test_start_date:test_end_date]
portfolio_return_long_test.name = "Weighted Long-term Portfolio"

benchmark_returns_long_test = benchmark_long_returns.loc[test_start_date:test_end_date]

# Replace the QuantStats report generation section with this:
print(f"\nGenerating Long-term QuantStats reports...")
print(f"Portfolio return length: {len(portfolio_return_long_test)}")
print(f"Benchmark return length: {len(benchmark_returns_long_test)}")

# Temporarily suppress warnings during report generation
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    qs.reports.html(
        portfolio_return_long_test,
        benchmark_returns_long_test,
        rf=risk_free_rate,
        figsize=(8, 5),
        output=f'portfolios/portfolio_vs_benchmark_long_term-{datetime.date(end_date)}.html',
        title=f'Long-term Portfolio vs {benchmark_long} (Benchmark) - Test Period',
        benchmark_title=f'{benchmark_long}',
        download_filename=f'portfolio_vs_{benchmark_long}_long_term.html'
    )
    
    qs.reports.full(
        portfolio_return_long_test,
        benchmark_returns_long_test,
        rf=risk_free_rate,
        figsize=(8, 5),
        title=f'Long-term Portfolio vs {benchmark_long} - Test Period',
        benchmark_title=f'{benchmark_long}'
    )


LONG-TERM PORTFOLIO BACKTESTING
Long-term Portfolio weights:
MMC: 0.3181
AMAT: 0.2289
TMUS: 0.2237
MRK: 0.1453
APO: 0.0840


100%|██████████| 2/2 [00:00<00:00,  7.84it/s]



Long-term Backtest Results:
Stat                 Long-term Portfolio    YYY
-------------------  ---------------------  ----------
Start                2023-06-01             2023-06-01
End                  2025-06-04             2025-06-04
Risk-free rate       4.24%                  4.24%

Total Return         29.04%                 21.14%
Daily Sharpe         0.64                   0.56
Daily Sortino        1.04                   0.80
CAGR                 13.53%                 10.01%
Max Drawdown         -13.94%                -14.44%
Calmar Ratio         0.97                   0.69

MTD                  0.86%                  0.70%
3m                   -2.88%                 0.35%
6m                   -6.06%                 -1.79%
YTD                  1.51%                  3.13%
1Y                   2.86%                  6.77%
3Y (ann.)            13.53%                 10.01%
5Y (ann.)            -                      -
10Y (ann.)           -                      -
Since Incep

                           YYY         Strategy
-------------------------  ----------  ----------
Start Period               2023-06-02  2023-06-02
End Period                 2025-06-04  2025-06-04
Risk-Free Rate             4.24%       4.24%
Time in Market             90.0%       100.0%

Cumulative Return          24.18%      30.11%
CAGR﹪                     0.03%       0.04%

Sharpe                     0.65        0.65
Prob. Sharpe Ratio         68.08%      62.99%
Smart Sharpe               0.6         0.6
Sortino                    0.87        0.92
Smart Sortino              0.81        0.85
Sortino/√2                 0.62        0.65
Smart Sortino/√2           0.57        0.6
Omega                      1.12        1.12

Max Drawdown               -14.74%     -15.66%
Longest DD Days            122         184
Volatility (ann.)          11.2%       15.88%
R^2                        0.38        0.38
Information Ratio          0.01        0.01
Calmar                     0.52        0.6

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-12-03,2025-04-08,2025-06-04,184,-15.658337,-14.391188
2,2024-07-17,2024-08-05,2024-10-11,87,-9.844154,-9.279887
3,2023-09-15,2023-10-27,2023-11-09,56,-6.535265,-5.899937
4,2024-10-15,2024-10-31,2024-11-29,46,-5.160199,-5.000851
5,2023-12-14,2023-12-20,2024-01-11,29,-3.583005,-2.526976


#### **Short-Term Portfolio Performance**

In [ ]:
print("\n" + "="*60)
print("SHORT-TERM PORTFOLIO BACKTESTING")
print("="*60)

# Create weights dictionary for short-term portfolio
weights_short = {}
for _, row in portfolio_short_df.iterrows():
    ticker = row['Ticker']
    weight = row['Weight']
    weights_short[ticker] = weight

print("Short-term Portfolio weights:")
for ticker, weight in weights_short.items():
    print(f"{ticker}: {weight:.4f}")

# Create test set for short-term data (instead of using long-term test_set)
total_rows_short = len(stock_short_quotes)
train_size_short = int(0.8 * total_rows_short)
test_set_short = stock_short_quotes.iloc[train_size_short:]

# Get test period dates for short-term
test_start_date_short = test_set_short.index[0]
test_end_date_short = test_set_short.index[-1]

print(f"Short-term test period: {test_start_date_short} to {test_end_date_short}")

# Create price data for bt (using short-term test set)
all_quotes_short = test_set_short.copy()
all_quotes_short[benchmark_short] = benchmark_short_quotes[benchmark_short].loc[test_start_date_short:test_end_date_short]

print(f"Short-term quotes columns: {list(all_quotes_short.columns)}")
print(f"Short-term portfolio tickers: {portfolio_short_tickers}")

# Create bt strategies for short-term
portfolio_strategy_short = bt.Strategy('Short-term Portfolio', 
                                      [
                                          bt.algos.RunWeekly(),  # More frequent rebalancing for short-term
                                          bt.algos.SelectAll(),
                                          bt.algos.WeighSpecified(**weights_short),       
                                          bt.algos.Rebalance()
                                      ])

benchmark_strategy_short = bt.Strategy(f'{benchmark_short}',
                                      [
                                          bt.algos.RunWeekly(),
                                          bt.algos.SelectThese([benchmark_short]),
                                          bt.algos.WeighEqually(),
                                          bt.algos.Rebalance()
                                      ])

# Create backtests for short-term
portfolio_test_short = bt.Backtest(portfolio_strategy_short, all_quotes_short[portfolio_short_tickers])
benchmark_test_short = bt.Backtest(benchmark_strategy_short, all_quotes_short[[benchmark_short]])

# Run backtests for short-term
result_short = bt.run(portfolio_test_short, benchmark_test_short)
result_short.set_riskfree_rate(risk_free_rate)

print("\nShort-term Backtest Results:")
result_short.display()

# Plot short-term results
result_short.plot(figsize=(12, 8),
                 ylabel='Cumulative Returns',
                 title='Short-term Portfolio vs Benchmark Performance',
                 legend=True,
                 grid=True)

# Generate short-term QuantStats reports (using short-term test period)
portfolio_return_short_test = portfolio_short_return.loc[test_start_date_short:test_end_date_short]
portfolio_return_short_test.name = "Weighted Short-term Portfolio"

benchmark_returns_short_test = benchmark_short_returns.loc[test_start_date_short:test_end_date_short]

print(f"\nGenerating Short-term QuantStats reports...")
print(f"Portfolio return length: {len(portfolio_return_short_test)}")
print(f"Benchmark return length: {len(benchmark_returns_short_test)}")

# Temporarily suppress warnings during report generation
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    qs.reports.html(
        portfolio_return_short_test,
        benchmark_returns_short_test,
        rf=risk_free_rate,
        figsize=(8, 5),
        output=f'portfolios/portfolio_vs_benchmark_short_term-{datetime.date(end_date)}.html',
        title=f'Short-term Portfolio vs {benchmark_short} (Benchmark) - Test Period',
        benchmark_title=f'{benchmark_short}',
        download_filename=f'portfolio_vs_{benchmark_short}_short_term.html'
    )
    
    qs.reports.full(
        portfolio_return_short_test,
        benchmark_returns_short_test,
        rf=risk_free_rate,
        figsize=(8, 5),
        title=f'Short-term Portfolio vs {benchmark_short} - Test Period',
        benchmark_title=f'{benchmark_short}'
    )


SHORT-TERM PORTFOLIO BACKTESTING
Short-term Portfolio weights:
MSFT: 0.5000
NOC: 0.5000
Short-term test period: 2023-06-02 00:00:00 to 2025-06-04 00:00:00
Short-term quotes columns: ['MSFT', 'NOC', 'DSI']
Short-term portfolio tickers: ['MSFT', 'NOC']


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.23it/s]



Short-term Backtest Results:
Stat                 Short-term Portfolio    DSI
-------------------  ----------------------  ----------
Start                2023-06-01              2023-06-01
End                  2025-06-04              2025-06-04
Risk-free rate       4.24%                   4.24%

Total Return         29.85%                  40.70%
Daily Sharpe         0.63                    0.84
Daily Sortino        1.04                    1.39
CAGR                 13.88%                  18.52%
Max Drawdown         -12.70%                 -20.58%
Calmar Ratio         1.09                    0.90

MTD                  1.05%                   0.92%
3m                   13.42%                  5.84%
6m                   5.07%                   -2.65%
YTD                  8.96%                   1.70%
1Y                   13.80%                  11.89%
3Y (ann.)            13.88%                  18.52%
5Y (ann.)            -                       -
10Y (ann.)           -               

                           DSI         Strategy
-------------------------  ----------  ----------
Start Period               2023-06-02  2023-06-02
End Period                 2025-06-04  2025-06-04
Risk-Free Rate             4.24%       4.24%
Time in Market             100.0%      100.0%

Cumulative Return          38.84%      24.54%
CAGR﹪                     0.04%       0.03%

Sharpe                     0.81        0.5
Prob. Sharpe Ratio         70.44%      54.87%
Smart Sharpe               0.8         0.5
Sortino                    1.17        0.72
Smart Sortino              1.16        0.71
Sortino/√2                 0.82        0.51
Smart Sortino/√2           0.82        0.5
Omega                      1.1         1.1

Max Drawdown               -21.06%     -14.62%
Longest DD Days            132         254
Volatility (ann.)          17.04%      16.2%
R^2                        0.31        0.31
Information Ratio          -0.02       -0.02
Calmar                     0.57        0.54


None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-09-24,2025-04-22,2025-06-04,254,-14.619161,-13.172399
2,2023-07-26,2023-09-26,2023-10-13,80,-8.774791,-8.472338
3,2024-04-01,2024-04-18,2024-05-20,50,-4.722896,-4.283867
4,2024-05-23,2024-05-30,2024-07-25,64,-4.639313,-4.473687
5,2023-12-14,2023-12-20,2024-01-11,29,-3.467673,-3.391322


#### **Combined Summary**

In [ ]:
print("\n" + "="*60)
print("PORTFOLIO COMPARISON SUMMARY")
print("="*60)

# Calculate metrics with proper error handling
try:
    portfolio_sharpe_long = qs.stats.sharpe(portfolio_return_long_test)
    if isinstance(portfolio_sharpe_long, pd.Series):
        portfolio_sharpe_long = portfolio_sharpe_long.iloc[0]
except:
    portfolio_sharpe_long = 0.0

try:
    benchmark_sharpe_long = qs.stats.sharpe(benchmark_returns_long_test)
    if isinstance(benchmark_sharpe_long, pd.Series):
        benchmark_sharpe_long = benchmark_sharpe_long.iloc[0]
except:
    benchmark_sharpe_long = 0.0

try:
    portfolio_total_long = qs.stats.comp(portfolio_return_long_test)
    if isinstance(portfolio_total_long, pd.Series):
        portfolio_total_long = portfolio_total_long.iloc[0]
except:
    portfolio_total_long = 0.0

try:
    benchmark_total_long = qs.stats.comp(benchmark_returns_long_test)
    if isinstance(benchmark_total_long, pd.Series):
        benchmark_total_long = benchmark_total_long.iloc[0]
except:
    benchmark_total_long = 0.0

print(f"\nLong-term Portfolio vs {benchmark_long}:")
print(f"  Portfolio Sharpe: {portfolio_sharpe_long:.4f}")
print(f"  Benchmark Sharpe: {benchmark_sharpe_long:.4f}")
print(f"  Portfolio Total Return: {portfolio_total_long:.4f}")
print(f"  Benchmark Total Return: {benchmark_total_long:.4f}")

# Calculate metrics for short-term with proper error handling
try:
    portfolio_sharpe_short = qs.stats.sharpe(portfolio_return_short_test)
    if isinstance(portfolio_sharpe_short, pd.Series):
        portfolio_sharpe_short = portfolio_sharpe_short.iloc[0]
except:
    portfolio_sharpe_short = 0.0

try:
    benchmark_sharpe_short = qs.stats.sharpe(benchmark_returns_short_test)
    if isinstance(benchmark_sharpe_short, pd.Series):
        benchmark_sharpe_short = benchmark_sharpe_short.iloc[0]
except:
    benchmark_sharpe_short = 0.0

try:
    portfolio_total_short = qs.stats.comp(portfolio_return_short_test)
    if isinstance(portfolio_total_short, pd.Series):
        portfolio_total_short = portfolio_total_short.iloc[0]
except:
    portfolio_total_short = 0.0

try:
    benchmark_total_short = qs.stats.comp(benchmark_returns_short_test)
    if isinstance(benchmark_total_short, pd.Series):
        benchmark_total_short = benchmark_total_short.iloc[0]
except:
    benchmark_total_short = 0.0

print(f"\nShort-term Portfolio vs {benchmark_short}:")
print(f"  Portfolio Sharpe: {portfolio_sharpe_short:.4f}")
print(f"  Benchmark Sharpe: {benchmark_sharpe_short:.4f}")
print(f"  Portfolio Total Return: {portfolio_total_short:.4f}")
print(f"  Benchmark Total Return: {benchmark_total_short:.4f}")

print(f"\nReports generated:")
print(f"  - portfolios/portfolio_vs_benchmark_long_term-{datetime.date(end_date)}.html")
print(f"  - portfolios/portfolio_vs_benchmark_short_term-{datetime.date(end_date)}.html")


PORTFOLIO COMPARISON SUMMARY

Long-term Portfolio vs YYY:
  Portfolio Sharpe: 0.9099
  Benchmark Sharpe: 1.0251
  Portfolio Total Return: 0.3011
  Benchmark Total Return: 0.2418

Short-term Portfolio vs DSI:
  Portfolio Sharpe: 0.7596
  Benchmark Sharpe: 1.0500
  Portfolio Total Return: 0.2454
  Benchmark Total Return: 0.3884

Reports generated:
  - portfolios/portfolio_vs_benchmark_long_term-2025-06-06.html
  - portfolios/portfolio_vs_benchmark_short_term-2025-06-06.html
